In [ ]:
"""
We will start by implementing the experiments conducted by Albert, Jeong, and Barabasi.
"""

In [13]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns

### Creating an ER Random Graph and a WS Small World Graph

In [21]:
n = 10000
m = 20000
k = 4
# p_ER = 0.01
p_WS = 0.2

ER = nx.gnm_random_graph(n, m)
# ER = nx.erdos_renyi_graph(n, p_ER)
WS = nx.watts_strogatz_graph(n, k, p_WS)
print(ER, WS)

Graph with 10000 nodes and 20000 edges Graph with 10000 nodes and 20000 edges


### Reading in the Electrical Grid Graph and Using that

In [22]:
def read_graph(filename):
    G = nx.Graph()
    array = np.loadtxt(filename, dtype=int)
    G.add_edges_from(array)
    return G

In [12]:
electrical_grid = read_graph('opsahl-powergrid/out.opsahl-powergrid')
print(electrical_grid)

Graph with 4941 nodes and 6594 edges
